<a href="https://colab.research.google.com/github/JustinPark845/Natural-Language-Processing-Final-Project/blob/main/SentimentEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Download all three sentiments
!pip install flair
!pip install spacy==3.4
!pip install spacytextblob
!pip install vaderSentiment
!pip install pysentiment2

In [ ]:
import flair
import spacy
from flair.models import TextClassifier
from flair.data import Sentence
from spacytextblob.spacytextblob import SpacyTextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pysentiment2 as ps

#Load classifier for English
flair_sentiment = TextClassifier.load('en-sentiment')
spacy_sentiment = spacy.load('en_core_web_sm')
spacy_sentiment.add_pipe('spacytextblob')
hiv4 = ps.HIV4()

In [ ]:
from operator import index
import csv

flair_corr = 0
spacy_corr = 0
vader_corr = 0
pysent2_corr = 0
invalid = 0
total = 0
count = 0
f = open("/content/drive/MyDrive/nlp_project/train.tsv", "r")
tsv_file = csv.reader(f, delimiter="\t")
for line in tsv_file:
  if count == 0:
    count += 1
    continue
  total += 1
  if int(line[3]) > 2:
     corr = 1
  elif int(line[3]) < 2:
     corr = -1
  else: 
     invalid += 1
     continue

  # Flair
  s = flair.data.Sentence(line[2])
  flair_sentiment.predict(s)
  try:
    total_sentiment = s.labels[0]
    assert total_sentiment.value in ['POSITIVE', 'NEGATIVE']
    val_f = 1 if total_sentiment.value == 'POSITIVE' else -1
    if val_f == corr:
      flair_corr +=1
  except IndexError:
    print("skipped")

  #Spacy
  text = spacy_sentiment(line[2])
  polarity_score = text._.polarity
  if polarity_score > 0:
    val_s = 1
  else:
    val_s = -1
  if val_s == corr:
    spacy_corr += 1

  #Vader
  vader_sent = SentimentIntensityAnalyzer()
  vader_result = vader_sent.polarity_scores(line[2])
  if vader_result['compound'] > 0.05:
    val_v = 1
  else:
    val_v = -1
  if val_v == corr:
    vader_corr += 1
  
  #PySentiment2
  tokens = hiv4.tokenize(n)
  val_py = -1
  score = hiv4.get_score(tokens)
  if score['Positive'] >= score['Negative']:
    val_py = 1
  if val_py == corr:
    pysent2_corr += 1
  print(total)

f.close()

In [ ]:
print("total", total-invalid)
print("flair accuracy is: ", flair_corr/(total - invalid))
print("spaCy accuracy is: ", spacy_corr/(total - invalid))
print("vader accuracy is: ", vader_corr/(total - invalid))
print("PySentiment2 accuracy is: ", pysent2_corr/(total - invalid))